## NCC 학습 데이터의 어노테이션을 추출
1번 데이터셋은 초록색 (0, 255, 0)  
2번 데이터셋은 붉은색 (120, 33, 33 정도)

In [1]:
import pandas as pd
import cv2
import os
from glob import glob
import pydicom as dicom
import numpy as np
from matplotlib import pyplot as plt
from IPython.core.debugger import set_trace

%matplotlib inline

In [2]:
ncc_base_path = "/home/huray/data/NCC_trainset"
dicom_base_path = os.path.join(ncc_base_path, "dicom")

img_save_path = dicom_base_path.replace('dicom', 'img_retinanet')

WIDTH = 1400
HEIGHT = 1750

In [3]:
import shutil
import os

files = sorted(glob("/home/huray/data/NCC_trainset/dicom/**/*.dcm", recursive=True))
print(files[:5])
print(len(files))

['/home/huray/data/NCC_trainset/dicom/abn_180116_1/18123764_0001.dcm', '/home/huray/data/NCC_trainset/dicom/abn_180116_1/18123764_0002.dcm', '/home/huray/data/NCC_trainset/dicom/abn_180116_1/18123764_0003.dcm', '/home/huray/data/NCC_trainset/dicom/abn_180116_1/18123764_0004.dcm', '/home/huray/data/NCC_trainset/dicom/abn_180116_1/30615287_0001.dcm']
3220


In [4]:
def detect_anno_set1(file_path):
    file_path = file_path.replace('.dcm', '.jpg')
    mask_arr = cv2.imread(file_path)
#     print(file_path)
    lower=np.array([0,252,0],np.uint8)
    upper=np.array([3,255,3],np.uint8)
    
    mask = cv2.inRange(mask_arr,lower,upper)
    
    (img_y, img_x, _) = mask_arr.shape
    
    scale_y = HEIGHT/img_y
    scale_x = WIDTH/img_x
    
    ret, thresh = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    _image, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    annotation_counter = 0
    
    mask_arr = cv2.resize(mask_arr, (0, 0), fx=scale_x, fy=scale_y)
    
    anno_list = []
    
    for index, c in enumerate(contours):
#         print('contour index: ', index)
#         if index >= 1:
#             print('more than one contour!!!!')
        x, y, w, h = cv2.boundingRect(c)
        x = int(x*scale_x)
        w = int(w*scale_x)
        y = int(y*scale_y)
        h = int(h*scale_y)
        anno_list.append((x,y,x+w,y+h))
#         cv2.rectangle(mask_arr, (x,y), (x+w,y+h), (255, 0, 255), 10)
        
#     plt.figure(figsize=(20, 20))
#     plt.imshow(mask_arr)
#     plt.title('image')
#     plt.show()

    return anno_list

In [5]:
def detect_anno_set2(file_path):
    file_path = file_path.replace('.dcm', '.jpg')
    mask_arr = cv2.imread(file_path)
#     print(file_path)
    
    # 1차 검출 -> 적당한 범위로 어노테이션을 찾아서(컨투어 여러개 검출됨) 사각형을 하나 그림
    lower=np.array([20,20,120],np.uint8)
    upper=np.array([60,60,200],np.uint8)
    mask = cv2.inRange(mask_arr,lower,upper)
    
    ret, thresh = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    _image, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    annotation_counter = 0
    
    for index, c in enumerate(contours):
        x, y, w, h = cv2.boundingRect(c)
        cv2.rectangle(mask_arr, (x,y), (x+w,y+h), (0, 255, 0), 8)
        
    # 2차 검출 -> 위에서 그린 사각형으로 정확한 어노테이션만 검출.
    lower=np.array([0,252,0],np.uint8)
    upper=np.array([3,255,3],np.uint8)
    mask = cv2.inRange(mask_arr,lower,upper)
        
    (img_y, img_x, _) = mask_arr.shape
    
    scale_y = HEIGHT/img_y
    scale_x = WIDTH/img_x    
        
    mask_arr = cv2.resize(mask_arr, (0, 0), fx=scale_x, fy=scale_y)
    
    ret, thresh = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    _image, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    annotation_counter = 0
    
    anno_list = []
    
    for index, c in enumerate(contours):
#         print('contour index: ', index)
#         if index >= 1:
#             print('more than one contour!!!!')
        x, y, w, h = cv2.boundingRect(c)
        x = int(x*scale_x)
        w = int(w*scale_x)
        y = int(y*scale_y)
        h = int(h*scale_y)
        anno_list.append((x,y,x+w,y+h))
#         cv2.rectangle(mask_arr, (x,y), (x+w,y+h), (255, 0, 255), 7)
#         print(x,y,w,h)
    
#     plt.figure(figsize=(8, 8))
#     plt.imshow(mask_arr)
#     plt.title('image')
#     plt.show()
    
    return anno_list

In [29]:
img_path = []
x1 = []
y1 = []
x2 = []
y2 = []
class_name = []

errored_data_counter = 0
files_with_wrong_size = []

# continuer = True

for mammo_path in files:
#     if '180116_2/33250647_0001' in mammo_path:
#         continuer = False
    
#     if continuer:
#         continue
    if not 'normal_180130' in mammo_path:
        continue

    print(mammo_path)
    mammo_dcm = dicom.read_file(mammo_path)
    
    if not 'normal_180130' in mammo_path and os.path.getsize(mammo_path)/(1024*1024) <= 1: # dcm파일이 1메가보다 작으면 pass
        files_with_wrong_size.append(mammo_path)
        print('DCM file is too small ' + mammo_path)
        continue
    
    try:
        mammo_arr = mammo_dcm.pixel_array
        mammo_arr = mammo_arr.astype(np.uint16)
    except (AttributeError, NotImplementedError) as e: # 종종 파일 자체가 문제가 있는 경우 있음.
        try:
            pixel_data = mammo_dcm[0x7fe0,0x0010].value # 파일 자체에 저장된 pixel_data값
            rows = mammo_dcm[0x0028, 0x0010].value # metadata로 들어있는 row
            cols = mammo_dcm[0x0028, 0x0011].value # metadata로 들어있는col

            mammo_arr = np.fromstring(pixel_data[:-1], dtype=np.uint16)
            mammo_arr = np.reshape(mammo_arr, (rows, cols))
        except ValueError:
            print('corrupted file: ' + mammo_path[:70])
            errored_data_counter += 1
            continue
        else:
            print('Attribute error" ' + mammo_path[:70])
    except Exception as e:
        print('different error: ' + mammo_path[:70])
        print(e)
        errored_data_counter += 1
        raise
        continue
        
    if len(mammo_arr.shape) == 3:
        print('RGB dicom!?')
        mammo_arr = cv2.cvtColor(mammo_arr, cv2.COLOR_BGR2GRAY)
        
    if mammo_arr.shape[0] <= 1800 and mammo_arr.shape[1] <= 1800:
        continue
        
    mammo_arr_final = (mammo_arr - np.amin(mammo_arr))/(np.amax(mammo_arr) - np.amin(mammo_arr)) * 255
    mammo_arr_final = mammo_arr_final.astype(np.uint8)
    mammo_arr_final = cv2.resize(mammo_arr_final, (WIDTH, HEIGHT))
    mammo_arr_final = np.asarray(np.dstack((mammo_arr_final, mammo_arr_final, mammo_arr_final)), dtype=np.uint8)

    anno_list = []
    
    if '180116_1' in mammo_path:
        anno_list = detect_anno_set1(mammo_path)
        data_sub_path = 'abn_180116_1'
    elif '180116_2' in mammo_path:
        anno_list = detect_anno_set2(mammo_path)
        data_sub_path = 'abn_180116_2'
    elif 'normal_180130' in mammo_path:
        anno_list = []
        data_sub_path = 'normal_180130'
    else:
        print('WRONG PATH!')
        raise
        
#     break
#     set_trace()
    
    _split = mammo_path.split('/')
    img_file_name = '{}'.format(_split[-1].replace('.dcm', ''))
    
    if len(anno_list) == 0:
        mammo_jpg_path = os.path.join(img_save_path, data_sub_path, img_file_name+'_0.jpg')
        cv2.imwrite(mammo_jpg_path, mammo_arr_final)
        
        img_path.append(mammo_jpg_path)
        x1.append('')
        y1.append('')
        x2.append('')
        y2.append('')
        class_name.append('')
    else:
        for i, anno_coord in enumerate(anno_list):
            mammo_jpg_path = os.path.join(img_save_path, data_sub_path, img_file_name+'_'+str(i)+'.jpg')
            cv2.imwrite(mammo_jpg_path, mammo_arr_final)

            img_path.append(mammo_jpg_path)
            x1.append(anno_coord[0])
            y1.append(anno_coord[1])
            x2.append(anno_coord[2])
            y2.append(anno_coord[3])
            class_name.append('M')
        
#     print(img_path, x1, y1, x2, y2)

/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_10.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_100.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1000.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1000
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1001.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1001
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1002.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1002
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1003.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1003
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1004.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal

/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1050.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1050
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1051.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1051
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1052.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1052
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1053.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1053
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1054.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1054
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1055.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1055
/home/huray/data/NCC_trainse

/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_110.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1100.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1100
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1101.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1101
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1102.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1102
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1103.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1103
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1104.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1104
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1105.dcm
corrupted file: /home/huray/data/NCC_trai

/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_115.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1150.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1151.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1152.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1153.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1154.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1155.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1156.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1157.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1158.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1159.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_116.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1160.dcm
/home/huray/data/NCC_trains

/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1249.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_125.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1250.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1251.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1252.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1253.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1254.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1255.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1256.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1257.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1258.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_1259.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_126.dcm
/home/huray/data/NCC_trains

...

/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_662.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_663.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_664.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_665.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_666.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_667.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_668.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_669.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_67.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_670.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_671.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_672.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_673.dcm
/home/huray/data/NCC_trainset/dicom/nor

/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_762.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_763.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_764.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_765.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_766.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_767.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_768.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_769.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_770.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_771.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_772.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_773.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_774.dcm
/home/huray/data/NCC_trainset/dicom/no

/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_866.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_867.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_868.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_869.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_87.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_870.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_871.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_872.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_873.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_874.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_875.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_876.dcm
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_877.dcm
/home/huray/data/NCC_trainset/dicom/nor

/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_940.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_940.
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_941.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_941.
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_942.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_942.
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_943.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_943.
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_944.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_944.
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_945.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_945.
/home/huray/data/NCC_trainset/dico

/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_990.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_990.
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_991.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_991.
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_992.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_992.
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_993.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_993.
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_994.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_994.
/home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_995.dcm
corrupted file: /home/huray/data/NCC_trainset/dicom/normal_180130/20180130_NORMAL_995.
/home/huray/data/NCC_trainset/dico

In [34]:
print("ERRORED DATA COUNT: ", errored_data_counter)
print("FILES WITH WRONG SIZE COUNT: ", len(files_with_wrong_size))
print("FILES WITH WRONG SIZE: ", files_with_wrong_size)

ERRORED DATA COUNT:  966
FILES WITH WRONG SIZE COUNT:  0
FILES WITH WRONG SIZE:  []


In [35]:
print(len(img_path), len(x1), len(y1), len(x2), len(y2), len(class_name))

1318 1318 1318 1318 1318 1318


In [36]:
df = pd.DataFrame({'img_path':img_path, 'x1':x1, 'y1':y1, 'x2':x2, 'y2':y2, 'class_name':class_name})
df = df[['img_path', 'x1', 'y1', 'x2', 'y2', 'class_name']]
print(len(df))

1318


In [37]:
df.head()

,img_path,x1,y1,x2,y2,class_name
0,/home/huray/data/NCC_trainset/img_retinanet/no...,,,,,
1,/home/huray/data/NCC_trainset/img_retinanet/no...,,,,,
2,/home/huray/data/NCC_trainset/img_retinanet/no...,,,,,
3,/home/huray/data/NCC_trainset/img_retinanet/no...,,,,,
4,/home/huray/data/NCC_trainset/img_retinanet/no...,,,,,


In [38]:
print(df['img_path'][0])
print(df['img_path'][1])
print(df['img_path'][2])
print(df['img_path'][3])

/home/huray/data/NCC_trainset/img_retinanet/normal_180130/20180130_NORMAL_1_0.jpg
/home/huray/data/NCC_trainset/img_retinanet/normal_180130/20180130_NORMAL_10_0.jpg
/home/huray/data/NCC_trainset/img_retinanet/normal_180130/20180130_NORMAL_100_0.jpg
/home/huray/data/NCC_trainset/img_retinanet/normal_180130/20180130_NORMAL_101_0.jpg


In [39]:
# df.to_csv('/home/huray/data/NCC_trainset/img_retinanet/data.csv', header=False, index=False)
df.to_csv('/home/huray/data/NCC_trainset/img_retinanet/data_normal.csv', header=False, index=False)